# Importowanie potrzebnych bibliotek

In [41]:
import numpy as np 
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from datetime import datetime
np.set_printoptions(precision = 12, suppress = True, linewidth = 150)
pd.options.display.float_format = '{:.6f}'. format


# Wczytanie pliku i przypisanie do zmiennej

In [15]:
raw_dataset = pd.read_excel('1000.xlsx')
raw_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 911 entries, 0 to 910
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   dlugosc_geogr               911 non-null    float64
 1   szerokosc_geogr             911 non-null    float64
 2   sredni_wiek_mieszkania      911 non-null    float64
 3   liczba_pokoi                911 non-null    float64
 4   liczba_sypialni             891 non-null    float64
 5   populacja                   911 non-null    float64
 6   gospodarstwa_domowe         911 non-null    float64
 7   sredni_przychód             911 non-null    float64
 8   srednia_wartosc_mieszkania  911 non-null    float64
dtypes: float64(9)
memory usage: 64.2 KB


In [3]:
dataset = raw_dataset
dataset.head()

,dlugosc_geogr,szerokosc_geogr,sredni_wiek_mieszkania,liczba_pokoi,liczba_sypialni,populacja,gospodarstwa_domowe,sredni_przychód,srednia_wartosc_mieszkania
0,-118.420000,34.170000,31.000000,2235.000000,363.000000,914.000000,370.000000,6.135900,359700.000000
1,-118.420000,34.160000,28.000000,4664.000000,1040.000000,1963.000000,961.000000,3.902800,367900.000000
2,-118.410000,34.160000,14.000000,577.000000,150.000000,372.000000,130.000000,4.187500,275000.000000
3,-118.420000,34.160000,25.000000,2769.000000,566.000000,1201.000000,545.000000,3.664100,386100.000000
4,-118.400000,34.170000,24.000000,4443.000000,1283.000000,2421.000000,1180.000000,2.265200,269200.000000


# Usuwanie pustych wartości z danych i sprawdzenie czy występują puste dane

In [16]:
dataset.dropna(inplace = True)
dataset.isnull().sum() / len(dataset)

dlugosc_geogr                0.000000
szerokosc_geogr              0.000000
sredni_wiek_mieszkania       0.000000
liczba_pokoi                 0.000000
liczba_sypialni              0.000000
populacja                    0.000000
gospodarstwa_domowe          0.000000
sredni_przychód              0.000000
srednia_wartosc_mieszkania   0.000000
dtype: float64

# Podział danych na zbiór treningowy i testowy

In [17]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

print(f'train_dataset length: {len(train_dataset)}')
print(f'test_dataset length: {len(test_dataset)}')

train_dataset length: 713
test_dataset length: 178


# Wyświetlenie podstawowych statystyk dla danych

In [18]:
train_stats = train_dataset.describe()
train_stats.pop('srednia_wartosc_mieszkania')
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
dlugosc_geogr,713.000000,-118.346788,0.129020,-118.660000,-118.440000,-118.320000,-118.250000,-118.160000
szerokosc_geogr,713.000000,34.112931,0.055257,34.020000,34.070000,34.100000,34.160000,34.220000
sredni_wiek_mieszkania,713.000000,36.061711,11.068307,6.000000,29.000000,36.000000,45.000000,52.000000
liczba_pokoi,713.000000,2291.207574,1534.080270,22.000000,1388.000000,1965.000000,2843.000000,17377.000000
liczba_sypialni,713.000000,615.767181,448.268243,6.000000,335.000000,502.000000,774.000000,4457.000000
populacja,713.000000,1592.399719,1013.521518,20.000000,907.000000,1371.000000,2019.000000,7450.000000
gospodarstwa_domowe,713.000000,574.848527,406.771420,6.000000,321.000000,478.000000,725.000000,4204.000000
sredni_przychód,713.000000,3.304649,1.476726,0.536000,2.208300,3.000000,4.115400,10.551100


# Wycięcie z danych kolumny, w której znajdują się przewidywane wartości

In [19]:
train_labels = train_dataset.pop('srednia_wartosc_mieszkania')
test_labels = test_dataset.pop('srednia_wartosc_mieszkania')


# Normalizacja danych

In [20]:
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [9]:
normed_train_data.isnull().sum()

dlugosc_geogr             0
szerokosc_geogr           0
sredni_wiek_mieszkania    0
liczba_pokoi              0
liczba_sypialni           0
populacja                 0
gospodarstwa_domowe       0
sredni_przychód           0
dtype: int64

In [21]:
normed_test_data = normed_test_data.values

# Zdefiniowanie funkcji budującej model

In [22]:
def build_model():
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=[len(train_dataset.keys())]))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))

    model.compile(optimizer='adam',
                  loss ='mse',
                  metrics = ['mae', 'mse'])
    return model

# Wczytanie bazy danych i zdefiniowanie funkcji logowania i rejestracji

In [23]:

users = pd.read_excel('lista.xlsx').drop(columns='Unnamed: 0') 

def logowanie():
   
    i = 0
    while i<3:    
        login = input("login: ")
        password = input("Hasło: ")  
        user = users.loc[login == users['login']]
       
        if len(user) == 1:     
            if password == str(user['password'].values[0]):
                print("Witaj {}\n".format(login))
                users['data_ost_logowania'].loc[login == users['login']] = datetime.now().strftime("%d-%m-%Y")
                users.to_excel('lista.xlsx')
                user = users.loc[login == users['login']]
                break
            else:
                print('nie udało się zalogować... sprawdź login i hasło')
        else:
            print('nie udało się zalogować... sprawdź login i hasło')
        i += 1
    if i == 3:
        print("konto zablokowane")
    return user


def rejestracja():
    x=0
    while True:
        login = input("Wybierz login ")
        for i in users['login']:
        
            if login == i:
                print("Podany login jest zajęty")
            
                break
            else:
                x=1
                continue               
        if x == 1:
            break
            
    password = str(input("Wybierz haslo "))
    print("Teraz zostaniesz poproszony o uzupełnienie swoich danych")
    name = str(input("Podaj swoje imie "))
    surname=(str(input("Podaj swoje nazwisko ")))
    row=len(users)+1
    users.loc[row,:] = [login, password, name, surname,'user', "NaN"]
    users.to_excel('lista.xlsx')

    print("Pomyślnie zarejestrowano!\nNowy użytkownik to: {}\nMożesz teraz:\n*Wrócić do ekranu startowego -1\n*Zalogować się -2".format(login))
    choice = int(input())
    if choice == 1:
        print("Przenoszę Cie do ekranu głównego")
    elif choice ==2:
        logowanie()
    else:
        print("Nie ma takiej opcji")



# Program z funkcjonalnością budowy i trenowania modelu

In [53]:
model_run = False
while True:   
    print("1-Zarejestruj się\n2-Zaloguj sie\n3-Zbuduj model i trenuj sieć neuronową\n4-Opcje modelu\n5-Zakończ program")
    choice = input("Wybierz opcje 1-5\n")
    if choice == '1':
        rejestracja()
    elif choice == '2':
        logowanie()
        break
    elif choice =='3':
        model_run = True
        model = build_model()
        model.fit(normed_train_data, train_labels.values, epochs = 150, validation_split = 0.2, verbose = 1, batch_size = 30)
        test_predictions = model.predict(normed_test_data).flatten()
        pred = pd.DataFrame(test_labels)
        pred['predictions']= test_predictions
    elif choice =='4':
        if model_run == True:
            print("Wybierz opcje zwiazana z modelem:\n1-Wyswietl histogram dla danych wejsciowych\n2-Wyswietl wykres dopasowania\n3-Wyświetl model\n4-Wyświetl porównanie prognozy do danych rzeczywistych")
            choice = input()
            if choice == '1':
                hist = px.histogram(dataset, x='srednia_wartosc_mieszkania')
                hist.show()
            elif choice =='2':
                wykres = px.scatter(pred, 'srednia_wartosc_mieszkania','predictions')
                wykres.add_trace(go.Scatter(x=[0, 500000], y=[0, 500000], mode='lines'))
                wykres.show()
            elif choice =='3':
                model.summary()
            elif choice == '4':
                test_predictions = model.predict(normed_test_data).flatten()
                pred = pd.DataFrame(test_labels)
                pred['Prognozowana_wartosc']= test_predictions
                pred['Różnica']=pred.srednia_wartosc_mieszkania - pred.Prognozowana_wartosc
                print(pred.head(10))
            
            else:
                print("Nie ma takiej opcji!")
        else:
            print("Model nie został jeszcze przetestowany! Spróbuj funkcji Zbuduj model i trenuj sieć neuronową")
            continue
    elif choice == '5':
        print("zakończono program")
        break
    else:
        print("Nie ma takiej opcji!")
        continue

1-Zarejestruj się
2-Zaloguj sie
3-Zbuduj model i trenuj sieć neuronową
4-Opcje modelu
5-Zakończ program
Wybierz opcje 1-5
4
Model nie został jeszcze przetestowany! Spróbuj funkcji Zbuduj model i trenuj sieć neuronową
1-Zarejestruj się
2-Zaloguj sie
3-Zbuduj model i trenuj sieć neuronową
4-Opcje modelu
5-Zakończ program
Wybierz opcje 1-5
5
zakończono program


# Wyświetlenie bazy danych

In [30]:
users = pd.read_excel('lista.xlsx').drop(columns='Unnamed: 0')
users

,login,password,imię,nazwisko,rola,data_ost_logowania
0,kowal,abcd,Jan,Kowalski,admin,31-01-2022
1,ola,nowak,Aleksandra,Nowak,admin,01-02-2022
2,pwolski,piotr1,Piotr,Wolski,moderator,27-01-2022
3,nitek,niebieski,Sławomir,Nitecki,user,29-01-2022
4,wojcik,2334,Edward,Wójcik,user,NaN
5,karolina,osiem,Karolina,Zamojska,user,27-01-2022
6,admin1,haslo1,Mateusz,Testowy,user,NaN
7,admin,haslo,Mateusz,Testowy,user,31-01-2022
8,wiktoria,wiki,Wiktoria,Nowak,user,NaN
9,Wiktoria,wiki,Wiktoria,Nowak,user,31-01-2022
